In [89]:
import pandas as pd
import re

from subimage_creation import SubimageCreator

import os
import shutil

### Chargement donnée csv

In [78]:
chemin_csv = "C:/Users/33783/OneDrive/Documents/AgroParisTech/3eme_annee/iodaa/Fil_rouge/images/BoundingBoxDJI0099et100/SubimagesDJI0099_300_400_Results/vott-csv-export/DJI_0099_Cirses-export.csv"

donnees_cvs = pd.read_csv(chemin_csv)

print(donnees_cvs.head())

                           image        xmin        ymin        xmax  \
0      Normal_DJI_0099_0_800.jpg   44.414894    2.482272   85.549651   
1  Normal_DJI_0099_1200_2800.jpg   75.265957   22.695038  112.854621   
2  Normal_DJI_0099_1200_2800.jpg  203.989362   68.439719  245.478723   
3  Normal_DJI_0099_1200_2800.jpg  263.209242  129.787234  295.124135   
4  Normal_DJI_0099_1200_2800.jpg  294.414894  142.907812  318.173770   

         ymax  label  
0   39.007098  cirse  
1   54.255319  cirse  
2  105.319149  cirse  
3  158.510638  cirse  
4  165.248238  cirse  


### Récupération nom d'imagettes (100,100) contenant des bounding box

on créé des sous rectangle de ceux des bounding box faites pour considérer qu'une images est positive on considère qu'un angle au moins de ce sous rectangle est dans l'images.

pb : on ne peut pas discriminer les imagettes avec cirses uniquement et celles avec cirses + autres plantes

In [103]:
images = donnees_cvs['image']


nom_images = re.findall(r'Normal_DJI_\d{4}',images[0])
list_nom_images = []
nb_meme_images = []
cpt = 0
X = []
Y = []

for i in images:
    
    match = re.match(r'Normal_(DJI_\d{4})_(\d{1,4})_(\d{1,4})',i)
    nom = match.group(1)

    Y.append(int(match.group(2)))
    X.append(int(match.group(3)))
    
    if nom == nom_images : 
        cpt += 1
    
    else :
        nom_images = nom
        list_nom_images.append(nom_images)
        if cpt != 0:
            nb_meme_images.append(cpt)
        
        cpt = 1


nb_meme_images.append(cpt)

print(list_nom_images,nb_meme_images)
print(X)
print(Y)



['DJI_0099', 'DJI_0100'] [99, 28]
[800, 2800, 2800, 2800, 2800, 2800, 2800, 2800, 2800, 2800, 3200, 3600, 3600, 2800, 3200, 3200, 3200, 3200, 3200, 3600, 2400, 3200, 3200, 3200, 3200, 3200, 3200, 3200, 3200, 3200, 2800, 2800, 2800, 2800, 2800, 2800, 2800, 2400, 2400, 2400, 2400, 1600, 1600, 2000, 3600, 3600, 3600, 3600, 3600, 3600, 2800, 3200, 3200, 3200, 3200, 3200, 3200, 3200, 3200, 3200, 2000, 2000, 2400, 2800, 3200, 3200, 3200, 3200, 3200, 3200, 3200, 3600, 3600, 2400, 3200, 3200, 2800, 2800, 3600, 3200, 3600, 2800, 2800, 2800, 2800, 3200, 3600, 3600, 3600, 2400, 2400, 2400, 2400, 2400, 2400, 2800, 2800, 2800, 2800, 800, 3600, 3600, 3200, 3200, 3200, 2400, 2800, 2800, 2800, 2800, 2800, 2800, 2800, 3200, 3200, 3200, 3200, 400, 3600, 3600, 3600, 3600, 3200, 2800, 3600, 3600, 3600]
[0, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1800, 1800, 1800, 1800, 1800, 1800, 1800, 1800, 1800, 1800, 1800, 1800, 1800, 1800, 1800

In [104]:
x1 = donnees_cvs['xmin']
x2 = donnees_cvs['xmax']
y1 = donnees_cvs['ymin']
y2 = donnees_cvs['ymax']

In [105]:
ligne_image = 0

positive = []

for k in range (len(list_nom_images)):
    nom_images = list_nom_images[k]

    for l in range (nb_meme_images[k]):
        ligne = ligne_image + l

        largeur_4 = (x2[ligne] - x1[ligne])/4
        hauteur_4 = (y2[ligne] - y1[ligne])/4
        x1_bis = X[ligne] + x1[ligne] + largeur_4
        x2_bis = X[ligne] + x2[ligne] - largeur_4
        y1_bis = Y[ligne] + y1[ligne] + hauteur_4
        y2_bis = Y[ligne] + y2[ligne] - hauteur_4

        x1_bis = round(x1_bis // 100 * 100)
        x2_bis = round(x2_bis // 100 * 100)
        y1_bis = round(y1_bis // 100 * 100)
        y2_bis = round(y2_bis // 100 * 100)

        nom_image_positive = nom_images+"_"+str(y1_bis)+"_"+str(x1_bis)+".jpg"
        positive.append(nom_image_positive)

        if x2_bis != x1_bis and y2_bis != y1_bis :
            nom_image_positive = nom_images+"_"+str(y2_bis)+"_"+str(x2_bis)+".jpg"
            positive.append(nom_image_positive)
        elif x2_bis != x1_bis :
            nom_image_positive = nom_images+"_"+str(y1_bis)+"_"+str(x2_bis)+".jpg"
            positive.append(nom_image_positive)
        elif y2_bis != y1_bis :
            nom_image_positive = nom_images+"_"+str(y2_bis)+"_"+str(x1_bis)+".jpg"
            positive.append(nom_image_positive) 

        
    ligne += nb_meme_images[k]

positive = list(set(positive))
print(positive)

# if 'Normal_DJI_0099_0_900.jpg' in positive :
# if 'Normal_DJI_0099_0_800.jpg' in positive :
#     print('True')
# else :
#     print('False')


['DJI_0099_1300_3000.jpg', 'DJI_0099_1300_3700.jpg', 'DJI_0099_1700_2900.jpg', 'DJI_0100_0_800.jpg', 'DJI_0099_1900_3400.jpg', 'DJI_0099_1400_3100.jpg', 'DJI_0099_2700_3000.jpg', 'DJI_0100_1700_2900.jpg', 'DJI_0099_2700_3500.jpg', 'DJI_0099_800_3600.jpg', 'DJI_0100_1300_3000.jpg', 'DJI_0100_1300_2900.jpg', 'DJI_0100_1700_3000.jpg', 'DJI_0099_2200_3600.jpg', 'DJI_0099_1900_3300.jpg', 'DJI_0099_1100_2400.jpg', 'DJI_0100_1200_3700.jpg', 'DJI_0100_1900_3400.jpg', 'DJI_0099_2000_2900.jpg', 'DJI_0099_500_3200.jpg', 'DJI_0099_800_3200.jpg', 'DJI_0099_2400_2200.jpg', 'DJI_0100_1800_3400.jpg', 'DJI_0099_2200_2200.jpg', 'DJI_0099_2400_3100.jpg', 'DJI_0099_2700_3800.jpg', 'DJI_0099_1800_3500.jpg', 'DJI_0099_700_3600.jpg', 'DJI_0099_1400_3300.jpg', 'DJI_0099_2900_3300.jpg', 'DJI_0099_1000_2700.jpg', 'DJI_0099_2100_2200.jpg', 'DJI_0100_1200_3000.jpg', 'DJI_0099_1900_3000.jpg', 'DJI_0099_1900_2900.jpg', 'DJI_0099_700_2900.jpg', 'DJI_0099_1900_3100.jpg', 'DJI_0100_1700_3500.jpg', 'DJI_0100_1900_3500.

### Création des sous_images

In [92]:
for nom in list_nom_images : 

    print(nom)
    image_path = "C:/Users/33783/OneDrive/Documents/AgroParisTech/3eme_annee/iodaa/Fil_rouge/images/02_02_2023_100A/"+nom+".JPG" 
    sushi = SubimageCreator(image_path, size=(100, 100))
    sushi.cut()
    sushi.rebuild()

Normal_DJI_0099


AttributeError: 'NoneType' object has no attribute 'shape'

### Déplacement des sous_images dans des dossiers Positive/Negative

In [106]:
chemin_imagette = "C:/Users/33783/OneDrive/Documents/AgroParisTech/3eme_annee/iodaa/Fil_rouge/images/convo/Subimages"

chemin_annot_convo = "C:/Users/33783/OneDrive/Documents/AgroParisTech/3eme_annee/iodaa/Fil_rouge/images/convo/Annotee"

dossier_positif = os.path.join(chemin_annot_convo, 'Positive')
if not os.path.exists(dossier_positif):
    os.makedirs(dossier_positif)

# Créez le sous-dossier "negatif" s'il n'existe pas
dossier_negatif = os.path.join(chemin_annot_convo, 'Negative')
if not os.path.exists(dossier_negatif):
    os.makedirs(dossier_negatif)




In [107]:
print(positive)

['DJI_0099_1300_3000.jpg', 'DJI_0099_1300_3700.jpg', 'DJI_0099_1700_2900.jpg', 'DJI_0100_0_800.jpg', 'DJI_0099_1900_3400.jpg', 'DJI_0099_1400_3100.jpg', 'DJI_0099_2700_3000.jpg', 'DJI_0100_1700_2900.jpg', 'DJI_0099_2700_3500.jpg', 'DJI_0099_800_3600.jpg', 'DJI_0100_1300_3000.jpg', 'DJI_0100_1300_2900.jpg', 'DJI_0100_1700_3000.jpg', 'DJI_0099_2200_3600.jpg', 'DJI_0099_1900_3300.jpg', 'DJI_0099_1100_2400.jpg', 'DJI_0100_1200_3700.jpg', 'DJI_0100_1900_3400.jpg', 'DJI_0099_2000_2900.jpg', 'DJI_0099_500_3200.jpg', 'DJI_0099_800_3200.jpg', 'DJI_0099_2400_2200.jpg', 'DJI_0100_1800_3400.jpg', 'DJI_0099_2200_2200.jpg', 'DJI_0099_2400_3100.jpg', 'DJI_0099_2700_3800.jpg', 'DJI_0099_1800_3500.jpg', 'DJI_0099_700_3600.jpg', 'DJI_0099_1400_3300.jpg', 'DJI_0099_2900_3300.jpg', 'DJI_0099_1000_2700.jpg', 'DJI_0099_2100_2200.jpg', 'DJI_0100_1200_3000.jpg', 'DJI_0099_1900_3000.jpg', 'DJI_0099_1900_2900.jpg', 'DJI_0099_700_2900.jpg', 'DJI_0099_1900_3100.jpg', 'DJI_0100_1700_3500.jpg', 'DJI_0100_1900_3500.

In [108]:
for fichier in os.listdir(chemin_imagette):
    chemin_fichier_source = os.path.join(chemin_imagette, fichier)
    print(fichier)
    # Vérifiez si le fichier est dans la liste des fichiers positifs
    if fichier in positive:
        # Déplacez le fichier dans le dossier "positif"
        shutil.move(chemin_fichier_source, os.path.join(dossier_positif, fichier))
        print(f"Fichier {fichier} déplacé vers le dossier 'positif'.")
    else:
        # Déplacez le fichier dans le dossier "negatif"
        shutil.move(chemin_fichier_source, os.path.join(dossier_negatif, fichier))
        print(f"Fichier {fichier} déplacé vers le dossier 'negatif'.")

DJI_0099_0_0.jpg
Fichier DJI_0099_0_0.jpg déplacé vers le dossier 'negatif'.
DJI_0099_0_100.jpg
Fichier DJI_0099_0_100.jpg déplacé vers le dossier 'negatif'.
DJI_0099_0_1000.jpg
Fichier DJI_0099_0_1000.jpg déplacé vers le dossier 'negatif'.
DJI_0099_0_1100.jpg
Fichier DJI_0099_0_1100.jpg déplacé vers le dossier 'negatif'.
DJI_0099_0_1200.jpg
Fichier DJI_0099_0_1200.jpg déplacé vers le dossier 'negatif'.
DJI_0099_0_1300.jpg
Fichier DJI_0099_0_1300.jpg déplacé vers le dossier 'negatif'.
DJI_0099_0_1400.jpg
Fichier DJI_0099_0_1400.jpg déplacé vers le dossier 'negatif'.
DJI_0099_0_1500.jpg
Fichier DJI_0099_0_1500.jpg déplacé vers le dossier 'negatif'.
DJI_0099_0_1600.jpg
Fichier DJI_0099_0_1600.jpg déplacé vers le dossier 'negatif'.
DJI_0099_0_1700.jpg
Fichier DJI_0099_0_1700.jpg déplacé vers le dossier 'negatif'.
DJI_0099_0_1800.jpg
Fichier DJI_0099_0_1800.jpg déplacé vers le dossier 'negatif'.
DJI_0099_0_1900.jpg
Fichier DJI_0099_0_1900.jpg déplacé vers le dossier 'negatif'.
DJI_0099_0_2